# Creación del modelo

In [2]:
import pandas as pd
import pickle
from scipy.stats import poisson


In [6]:
from google.colab import files
uploaded = files.upload()

Saving dict_table.pkl to dict_table.pkl


In [7]:
df_fixture = pd.read_csv('D_Mundiales_FIFA_fixture_limpio.csv')
df_fixture

,Local,Resultado,Visitante,Año
0,Mexico,Match 1,South Africa,2026
1,South Korea,Match 2,UEFA Path D winner,2026
2,UEFA Path D winner,Match 25,South Africa,2026
3,Mexico,Match 28,South Korea,2026
4,UEFA Path D winner,Match 53,Mexico,2026
...,...,...,...,...
99,Winner Match 95,Match 100,Winner Match 96,2026
100,Winner Match 97,Match 101,Winner Match 98,2026
101,Winner Match 99,Match 102,Winner Match 100,2026
102,Loser Match 101,Match 103,Loser Match 102,2026


In [8]:
import pandas as pd
import pickle
from scipy.stats import poisson
import re

def clean_team_name(team_name):
    # Elimina texto entre paréntesis y corchetes
    cleaned_name = re.sub(r'\(.*?\)|\[.*?\]', '', str(team_name))
    # Elimina espacios en blanco iniciales/finales
    return cleaned_name.strip()

dict_table = pickle.load(open('dict_table.pkl','rb'))
df_data_historica = pd.read_csv('Clean_Mundiales_FIFA_registro_historico_limpio.csv')
df_fixture = pd.read_csv('D_Mundiales_FIFA_fixture_limpio.csv')

df_fixture['Local'] = df_fixture['Local'].apply(clean_team_name)
df_fixture['Visitante'] = df_fixture['Visitante'].apply(clean_team_name)

#aplica la limpieza a los nombres de los equipos en dict_table para cada grupo
for group in dict_table:
    dict_table[group]['Team'] = dict_table[group]['Team'].apply(clean_team_name)

In [9]:
dict_table['Grupo A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Mexico,0,0,0,0,0,0,0,0
1,2,South Africa,0,0,0,0,0,0,0,0
2,3,South Korea,0,0,0,0,0,0,0,0
3,4,UEFA Path D winner,0,0,0,0,0,0,0,0


In [10]:
df_data_historica

,Local,Visitante,Año,Goles_Locales,Goles_Visitantes,Goles_Totales
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
923,England,France,2022,1,2,3
924,Argentina,Croatia,2022,3,0,3
925,France,Morocco,2022,2,0,2
926,Croatia,Morocco,2022,2,1,3


In [11]:
df_fixture

,Local,Resultado,Visitante,Año
0,Mexico,Match 1,South Africa,2026
1,South Korea,Match 2,UEFA Path D winner,2026
2,UEFA Path D winner,Match 25,South Africa,2026
3,Mexico,Match 28,South Korea,2026
4,UEFA Path D winner,Match 53,Mexico,2026
...,...,...,...,...
99,Winner Match 95,Match 100,Winner Match 96,2026
100,Winner Match 97,Match 101,Winner Match 98,2026
101,Winner Match 99,Match 102,Winner Match 100,2026
102,Loser Match 101,Match 103,Loser Match 102,2026


## Fuerza del equipo

In [12]:
# se divide el df en local y visitante
df_home = df_data_historica[['Local', 'Goles_Locales', 'Goles_Visitantes']]
df_away = df_data_historica[['Visitante', 'Goles_Locales', 'Goles_Visitantes']]


In [13]:
df_home.rename(columns={'Local':'equipo',
                        'Goles_Locales':'Goles_anotados',
                        'Goles_Visitantes': 'Goles_recibidos'}, inplace=True)
df_home

/tmp/ipython-input-2198903432.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_home.rename(columns={'Local':'equipo',


,equipo,Goles_anotados,Goles_recibidos
0,France,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
923,England,1,2
924,Argentina,3,0
925,France,2,0
926,Croatia,2,1


In [14]:
df_away.rename(columns={'Visitante':'equipo',
                        'Goles_Locales':'Goles_recibidos',
                        'Goles_Visitantes': 'Goles_anotados'}, inplace=True)
df_away

/tmp/ipython-input-2987974571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_away.rename(columns={'Visitante':'equipo',


,equipo,Goles_recibidos,Goles_anotados
0,Mexico,4,1
1,France,1,0
2,Mexico,3,0
3,France,1,0
4,Mexico,6,3
...,...,...,...
923,France,1,2
924,Croatia,3,0
925,Morocco,2,0
926,Morocco,2,1


In [15]:
#Concatenar los df
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('equipo').mean()
df_team_strength

,Goles_anotados,Goles_recibidos
equipo,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.752941,1.164706
Australia,0.850000,1.850000
Austria,1.576923,1.692308
...,...,...
Uruguay,1.553571,1.303571
Wales,0.625000,1.250000
West Germany,2.050847,1.254237


## Función de predicción de puntos

In [16]:
def predict_points(local, visitante):
    if local in df_team_strength.index and visitante in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[local,'Goles_anotados'] * df_team_strength.at[visitante,'Goles_recibidos']
        lamb_away = df_team_strength.at[visitante,'Goles_anotados'] * df_team_strength.at[local,'Goles_recibidos']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [17]:
home_points, away_points = predict_points('Argentina', 'Colombia')
print(home_points, away_points)

1.8464304127135762 0.9683542813242045


## Predicción del mundial

In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [19]:
df_fixture

,Local,Resultado,Visitante,Año
0,Mexico,Match 1,South Africa,2026
1,South Korea,Match 2,UEFA Path D winner,2026
2,UEFA Path D winner,Match 25,South Africa,2026
3,Mexico,Match 28,South Korea,2026
4,UEFA Path D winner,Match 53,Mexico,2026
5,South Africa,Match 54,South Korea,2026
6,Canada,Match 3,UEFA Path A winner,2026
7,Qatar,Match 8,Switzerland,2026
8,Switzerland,Match 26,UEFA Path A winner,2026
9,Canada,Match 27,Qatar,2026


In [20]:
df_fixture_group_72 = df_fixture[:72].copy()
df_fixture_32 = df_fixture[72:88].copy()
df_fixture_8 = df_fixture[88:96].copy()
df_fixture_4 = df_fixture[96:100].copy()
df_fixture_semifinal = df_fixture[100:102].copy()
df_fixture_tercer = df_fixture[102:103].copy()
df_fixture_final = df_fixture[103:].copy()

In [21]:
df_fixture_tercer

,Local,Resultado,Visitante,Año
102,Loser Match 101,Match 103,Loser Match 102,2026


In [22]:
dict_table.keys()

dict_keys(['Grupo A', 'Grupo B', 'Grupo C', 'Grupo D', 'Grupo E', 'Grupo F', 'Grupo G', 'Grupo H', 'Grupo I', 'Grupo J', 'Grupo K', 'Grupo L'])

In [23]:
for group in dict_table:
    print(dict_table[group]['Team'].values)

['Mexico' 'South Africa' 'South Korea' 'UEFA Path D winner']
['Canada' 'UEFA Path A winner' 'Qatar' 'Switzerland']
['Brazil' 'Morocco' 'Haiti' 'Scotland']
['United States' 'Paraguay' 'Australia' 'UEFA Path C winner']
['Germany' 'Curaçao' 'Ivory Coast' 'Ecuador']
['Netherlands' 'Japan' 'UEFA Path B winner' 'Tunisia']
['Belgium' 'Egypt' 'Iran' 'New Zealand']
['Spain' 'Cape Verde' 'Saudi Arabia' 'Uruguay']
['France' 'Senegal' 'IC Path 2 winner' 'Norway']
['Argentina' 'Algeria' 'Austria' 'Jordan']
['Portugal' 'IC Path 1 winner' 'Uzbekistan' 'Colombia']
['England' 'Croatia' 'Ghana' 'Panama']


In [24]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_72[df_fixture_group_72['Local'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['Local'], row['Visitante']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

/tmp/ipython-input-4189374926.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.26662824]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
/tmp/ipython-input-4189374926.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.21935563]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
/tmp/ipython-input-4189374926.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.3527937]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]

In [25]:
dict_table['Grupo K']

,Team,Pts
0,Portugal,2.0
1,Colombia,1.0
2,IC Path 1 winner,0.0
3,Uzbekistan,0.0


In [26]:
df_fixture_group_72

,Local,Resultado,Visitante,Año
0,Mexico,Match 1,South Africa,2026
1,South Korea,Match 2,UEFA Path D winner,2026
2,UEFA Path D winner,Match 25,South Africa,2026
3,Mexico,Match 28,South Korea,2026
4,UEFA Path D winner,Match 53,Mexico,2026
5,South Africa,Match 54,South Korea,2026
6,Canada,Match 3,UEFA Path A winner,2026
7,Qatar,Match 8,Switzerland,2026
8,Switzerland,Match 26,UEFA Path A winner,2026
9,Canada,Match 27,Qatar,2026


## Ronda dieciseisavos de final

In [27]:
dict_table

{'Grupo A':                  Team  Pts
 0        South Africa  3.0
 1              Mexico  3.0
 2         South Korea  2.0
 3  UEFA Path D winner  0.0,
 'Grupo B':                  Team  Pts
 0         Switzerland  5.0
 1              Canada  2.0
 2               Qatar  1.0
 3  UEFA Path A winner  0.0,
 'Grupo C':        Team  Pts
 0    Brazil  7.0
 1   Morocco  5.0
 2  Scotland  4.0
 3     Haiti  0.0,
 'Grupo D':                  Team  Pts
 0            Paraguay  4.0
 1       United States  3.0
 2           Australia  2.0
 3  UEFA Path C winner  0.0,
 'Grupo E':           Team  Pts
 0      Germany  4.0
 1      Ecuador  2.0
 2  Ivory Coast  2.0
 3      Curaçao  0.0,
 'Grupo F':                  Team  Pts
 0         Netherlands  5.0
 1               Japan  2.0
 2             Tunisia  1.0
 3  UEFA Path B winner  0.0,
 'Grupo G':           Team  Pts
 0      Belgium  7.0
 1         Iran  4.0
 2        Egypt  4.0
 3  New Zealand  2.0,
 'Grupo H':            Team  Pts
 0         Spain  4.0
 

In [28]:
for group in dict_table:
  group_winner = dict_table[group].loc[0, 'Team']
  runners_up = dict_table[group].loc[1, 'Team']
  df_fixture_32.replace({f'Winner Group {group[-1]}': group_winner,
                         f'Runner-up Group {group[-1]}': runners_up}, inplace = True)
df_fixture_32['winner'] = '?'
df_fixture_32

,Local,Resultado,Visitante,Año,winner
72,Mexico,Match 73,Canada,2026,?
73,Brazil,Match 76,Japan,2026,?
74,Germany,Match 74,3rd Group A/B/C/D/F,2026,?
75,Netherlands,Match 75,Morocco,2026,?
76,Ecuador,Match 78,Senegal,2026,?
77,France,Match 77,3rd Group C/D/F/G/H,2026,?
78,South Africa,Match 79,3rd Group C/E/F/H/I,2026,?
79,England,Match 80,3rd Group E/H/I/J/K,2026,?
80,Belgium,Match 82,3rd Group A/E/H/I/J,2026,?
81,Paraguay,Match 81,3rd Group B/E/F/I/J,2026,?


In [29]:
def get_winner(df_fixture_updated):
  for index, row in df_fixture_updated.iterrows():
    home = row['Local']
    away = row['Visitante']

    # Check if teams are actual teams or placeholders
    is_home_real_team = home in df_team_strength.index
    is_away_real_team = away in df_team_strength.index

    if is_home_real_team and is_away_real_team:
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        elif points_away > points_home:
            winner = away
        else: # In case of a draw in predicted points, home team wins
            winner = home
    elif is_home_real_team and not is_away_real_team:
        winner = home # Real team wins against placeholder
    elif not is_home_real_team and is_away_real_team:
        winner = away # Real team wins against placeholder
    else: # Both are placeholders, default to away as before
        winner = away
    df_fixture_updated.loc[index,'winner'] = winner
  return df_fixture_updated

get_winner(df_fixture_32)

,Local,Resultado,Visitante,Año,winner
72,Mexico,Match 73,Canada,2026,Mexico
73,Brazil,Match 76,Japan,2026,Brazil
74,Germany,Match 74,3rd Group A/B/C/D/F,2026,Germany
75,Netherlands,Match 75,Morocco,2026,Netherlands
76,Ecuador,Match 78,Senegal,2026,Ecuador
77,France,Match 77,3rd Group C/D/F/G/H,2026,France
78,South Africa,Match 79,3rd Group C/E/F/H/I,2026,South Africa
79,England,Match 80,3rd Group E/H/I/J/K,2026,England
80,Belgium,Match 82,3rd Group A/E/H/I/J,2026,Belgium
81,Paraguay,Match 81,3rd Group B/E/F/I/J,2026,Paraguay
